# Create requisition and aliquot datasets and export to CSV

### This notebook creates two files:
1. `lab_requisition_{project}_{timestamp}.csv`
2. `lab_aliquot_{project}_{timestamp}.csv`


### This notebook accesses mysql tables not models
This notebook accesses mysql tables directly and not through model classes. A complete project VENV is not required.

## Tunnel to DB

Open a tunnel to the live DB:

    ssh -f <user>@<server> -L5002:localhost:3306 -N

Change the mysql settings (e.g. in your `/etc/<project>/mysql.conf`):

    port: 5002
    name: <database name>
    user: <readonly user>
    password: <password>


## Usage:

* Run the notebook server in a valid Django VENV where `settings` points to the database.
* requirements: `pandas`, `numpy`, `edc_pdutils`, `edc_constants`. Optional: `edc_base`, `edc_lab`. See imports.
* review your database `settings`
* start the server using someithing like:

        python manage.py shell_plus --notebook --settings=edc_pdutils.settings
        
    where `--settings` points to the correct settings file. 
> **Note:** since you need to start the notebook server in the folder where your notebooks are, navigate to your folder and start the notebook server from that folder. Use the `--settings` parameter to use a simplified settings file in some other folder (or the current one) that points to your database. It is better not to use the production settings file since none of the dependencies are needed.

In [1]:
# imports
import sys
import pandas as pd
import numpy as np

from edc_pdutils import Database
from edc_pdutils.table_to_dataframe import Helper, TableToDataframe
from edc_base.model_mixins.constants import DEFAULT_BASE_FIELDS
from edc_lab.model_mixins.requisition import RequisitionStatusMixin
from edc_constants.constants import YES, NO, NEG, UNK
from datetime import datetime
from copy import copy

yes_no = {True: YES, False: NO}

In [2]:
# settings
date_format = '%Y-%m-%d %H:%M:%S.%f'
export_date_format = '%Y-%m-%d'
local_tz = 'Africa/Gaborone'
delimiter = ',' # '|'
start_date = None   # datetime(2013, 10, 1)
project = 'bcpp_clinic'  # for export file name

In [3]:
timestamp = datetime.today().strftime('%Y%m%d%H%M%S')
helper = Helper()

In [4]:
# df_subjects
df_subjects = TableToDataframe(
    table_name='bcpp_clinic_subject_subjectconsent',
    columns=['subject_identifier', 'gender', 'dob', 'consent_datetime', 'study_site']).dataframe

df_subjects['dob'] = helper.date_to_local_datetime(df_subjects['dob'])
df_subjects['consent_datetime'] = helper.to_local_datetime(df_subjects['consent_datetime'])

df_subjects = df_subjects.sort_values(['subject_identifier'])
df_subjects = df_subjects.drop_duplicates()
df_subjects = df_subjects.groupby(['subject_identifier', 'consent_datetime']).min().reset_index()
df_subjects['duplicate'] = df_subjects.duplicated() 
if len(df_subjects[df_subjects['duplicate']]) > 0:
    print('Consent has duplicates!')
    print(df_subjects[df_subjects['duplicate']])
else:
    df_subjects = df_subjects.drop(['duplicate'], axis=1)
df_subjects['consent_date'] = df_subjects['consent_datetime'].dt.normalize()
df_subjects.head()

,subject_identifier,consent_datetime,gender,dob,study_site,consent_date
0,066-21730001-2,2017-08-29 08:05:40+00:00,M,1990-06-02 00:00:00+00:00,21,2017-08-29 00:00:00+00:00
1,066-21730002-0,2017-08-29 09:14:17+00:00,F,1984-10-01 00:00:00+00:00,21,2017-08-29 00:00:00+00:00
2,066-21730003-8,2017-08-29 09:27:42+00:00,F,1982-06-08 00:00:00+00:00,21,2017-08-29 00:00:00+00:00
3,066-21730004-6,2017-09-06 07:22:27+00:00,F,1993-05-27 00:00:00+00:00,21,2017-09-06 00:00:00+00:00
4,066-21730005-3,2017-09-06 08:48:40+00:00,M,1982-08-25 00:00:00+00:00,21,2017-09-06 00:00:00+00:00


In [5]:
# df_visit
df_visit = TableToDataframe(
    table_name='bcpp_clinic_subject_subjectvisit',
    columns=['id', 'subject_identifier', 'report_datetime', 'visit_code']).dataframe
df_visit['report_datetime'] = helper.to_local_datetime(df_visit['report_datetime'])
df_visit = df_visit.rename(columns={'id': 'subject_visit_id', 'report_datetime': 'visit_datetime'})
df_visit = pd.merge(df_visit, df_subjects, on='subject_identifier', how='left')
df_visit['visit_date'] = df_visit['visit_datetime'].dt.normalize()
df_visit.head()

,subject_visit_id,subject_identifier,visit_datetime,visit_code,consent_datetime,gender,dob,study_site,consent_date,visit_date
0,14c19b10a6084176af57d469e80dc3b0,066-21730002-0,2017-08-29 09:18:08+00:00,C0,2017-08-29 09:14:17+00:00,F,1984-10-01 00:00:00+00:00,21,2017-08-29 00:00:00+00:00,2017-08-29 00:00:00+00:00
1,1b376d1b11e0409e9139090e305c1691,066-21730005-3,2017-09-06 09:00:35+00:00,C0,2017-09-06 08:48:40+00:00,M,1982-08-25 00:00:00+00:00,21,2017-09-06 00:00:00+00:00,2017-09-06 00:00:00+00:00
2,367b06e6b56046619fbe991942379001,066-31760002-2,2017-09-12 07:50:33+00:00,C0,2017-09-12 07:42:48+00:00,F,1972-06-12 00:00:00+00:00,31,2017-09-12 00:00:00+00:00,2017-09-12 00:00:00+00:00
3,37ed0953aa16459295345d1045002775,066-31760001-4,2017-09-06 08:14:48+00:00,C0,2017-09-06 07:58:34+00:00,F,1982-01-09 00:00:00+00:00,31,2017-09-06 00:00:00+00:00,2017-09-06 00:00:00+00:00
4,72adbc94177b48299b997c72a7f7a1f0,066-31760003-0,2017-09-13 08:11:15+00:00,C0,2017-09-13 07:55:20+00:00,F,1965-01-01 00:00:00+00:00,31,2017-09-13 00:00:00+00:00,2017-09-13 00:00:00+00:00


In [6]:
# df_requisition
columns = ['subject_visit_id', 'requisition_identifier', 'panel_name', 'requisition_datetime',
           'is_drawn', 'reason_not_drawn', 'drawn_datetime', 'specimen_type', 'study_site_name']
status_cols = [f.name for f in RequisitionStatusMixin._meta.get_fields()
               if f not in ['processed_datetime', 'packed_datetime', 'shipped_datetime']]
columns.extend([f.name for f in RequisitionStatusMixin._meta.get_fields()])

df_requisition = TableToDataframe(
    table_name='bcpp_clinic_subject_subjectrequisition',
    columns=columns).dataframe
df_requisition['requisition_datetime'] = helper.to_local_datetime(df_requisition['requisition_datetime'])
df_requisition['drawn_datetime'] = helper.to_local_datetime(df_requisition['drawn_datetime'])
df_requisition['received_datetime'] = helper.to_local_datetime(df_requisition['received_datetime'])

# df_requisition['processed_datetime'] = helper.to_local_datetime(df_requisition['processed_datetime'])
# df_requisition['packed_datetime'] = helper.to_local_datetime(df_requisition['packed_datetime'])
# df_requisition['shipped_datetime'] = helper.to_local_datetime(df_requisition['shipped_datetime'])

df_requisition['received'] = df_requisition['received'].map(yes_no, na_action='ignore')
df_requisition['processed'] = df_requisition['processed'].map(yes_no, na_action='ignore')
df_requisition['packed'] = df_requisition['packed'].map(yes_no, na_action='ignore')
df_requisition['shipped'] = df_requisition['shipped'].map(yes_no, na_action='ignore')
df_requisition = pd.merge(df_requisition, df_visit, on='subject_visit_id', how='left')

# fix reason_not_drawn
is_blank = (df_requisition.reason_not_drawn.notnull()) & (df_requisition.is_drawn == YES)
df_requisition.loc[is_blank, 'reason_not_drawn'] = np.nan
if start_date:
    df_requisition = df_requisition[df_requisition['visit_date'] >= start_date]
df_requisition.head()

,subject_visit_id,requisition_identifier,panel_name,requisition_datetime,is_drawn,reason_not_drawn,drawn_datetime,specimen_type,study_site_name,received,...,shipped_datetime,subject_identifier,visit_datetime,visit_code,consent_datetime,gender,dob,study_site,consent_date,visit_date
0,14c19b10a6084176af57d469e80dc3b0,73UUAD3,Viral Load,2017-08-29 09:19:13+00:00,Yes,NaN,2017-08-29 09:19:19+00:00,WB,lerala,Yes,...,None,066-21730002-0,2017-08-29 09:18:08+00:00,C0,2017-08-29 09:14:17+00:00,F,1984-10-01 00:00:00+00:00,21,2017-08-29 00:00:00+00:00,2017-08-29 00:00:00+00:00
1,37ed0953aa16459295345d1045002775,76HU8EE,Research Blood Draw,2017-09-06 08:16:47+00:00,Yes,NaN,2017-09-06 08:17:01+00:00,WB,mathangwane,Yes,...,None,066-31760001-4,2017-09-06 08:14:48+00:00,C0,2017-09-06 07:58:34+00:00,F,1982-01-09 00:00:00+00:00,31,2017-09-06 00:00:00+00:00,2017-09-06 00:00:00+00:00
2,f6ef553feb2744f59578bebd8c796397,73D784K,Viral Load,2017-08-29 08:10:21+00:00,Yes,NaN,2017-08-29 08:10:23+00:00,WB,lerala,Yes,...,None,066-21730001-2,2017-08-29 08:08:16+00:00,C0,2017-08-29 08:05:40+00:00,M,1990-06-02 00:00:00+00:00,21,2017-08-29 00:00:00+00:00,2017-08-29 00:00:00+00:00
3,1b376d1b11e0409e9139090e305c1691,73UPKBY,Research Blood Draw,2017-09-06 09:01:50+00:00,Yes,NaN,2017-09-06 09:01:58+00:00,WB,lerala,Yes,...,None,066-21730005-3,2017-09-06 09:00:35+00:00,C0,2017-09-06 08:48:40+00:00,M,1982-08-25 00:00:00+00:00,21,2017-09-06 00:00:00+00:00,2017-09-06 00:00:00+00:00
4,c07305835680417a81c82c7774158d92,73HT3NG,Research Blood Draw,2017-09-06 07:28:25+00:00,Yes,NaN,2017-09-06 10:12:44+00:00,WB,lerala,Yes,...,None,066-21730004-6,2017-09-06 07:27:43+00:00,C0,2017-09-06 07:22:27+00:00,F,1993-05-27 00:00:00+00:00,21,2017-09-06 00:00:00+00:00,2017-09-06 00:00:00+00:00


In [7]:
df_requisition.groupby('panel_name').size()

panel_name
Research Blood Draw    10
Viral Load              9
dtype: int64

In [8]:
df_requisition.groupby('study_site').size()

study_site
21    12
31     7
dtype: int64

In [9]:
# df_aliquot
df_aliquot = TableToDataframe(
    table_name='edc_lab_aliquot').dataframe
df_aliquot['aliquot_datetime'] = helper.to_local_datetime(df_aliquot['aliquot_datetime'])
df_aliquot['aliquot_date'] = df_aliquot['aliquot_datetime'].dt.normalize()

In [10]:
df_aliquot.groupby('hostname_created').size()

hostname_created
bcpp071    14
bcpp073    70
bcpp074    14
bcpp076    35
dtype: int64

In [11]:
# drop sys and other unwanted columns
base_cols = copy(DEFAULT_BASE_FIELDS)
base_cols.pop(base_cols.index('hostname_created'))
base_cols = base_cols + ['slug', 'comment', 'shipped']
df_aliquot = df_aliquot.drop(base_cols, axis=1)

In [12]:
def fix_parent_identifier(value):
    if value:
        value = value.replace('(', '').replace(')', '').replace('\'', '').replace(',', '')
    return value

# add column to indicate missing requisitions, remap columns
requisition_cols = ['requisition_identifier',
        'panel_name', 'requisition_datetime',
        'gender', 'dob', 'study_site', 'study_site_name', 'visit_code', 'visit_datetime', 'visit_date']
missing = pd.merge(df_aliquot, df_requisition[requisition_cols],
               on='requisition_identifier', how='left')
df_aliquot['missing_requisition'] = missing['panel_name'].isnull()
df_aliquot['is_primary'] = df_aliquot['is_primary'].map(yes_no, na_action='ignore')
df_aliquot['medium'] = df_aliquot['medium'].str.lower()
df_aliquot['parent_identifier'] = df_aliquot.apply(lambda row: fix_parent_identifier(row['parent_identifier']), axis=1)
df_aliquot = pd.merge(df_aliquot, df_requisition[requisition_cols], on='requisition_identifier', how='left')

columns = [col for col in list(df_aliquot.columns) if col not in [
    'hostname_created', 'subject_identifier', 'gender', 'dob', 'visit_code', 'visit_datetime', 'visit_date']]
columns.insert(0, 'visit_code')
columns.insert(0, 'visit_date')
columns.insert(0, 'visit_datetime')
columns.insert(0, 'dob')
columns.insert(0, 'gender')
columns.insert(0, 'subject_identifier')
columns.append('hostname_created')
df_aliquot = df_aliquot[columns]
df_aliquot.head()

,subject_identifier,gender,dob,visit_datetime,visit_date,visit_code,aliquot_identifier,parent_identifier,identifier_prefix,requisition_identifier,...,aliquot_type,alpha_code,numeric_code,aliquot_date,missing_requisition,panel_name,requisition_datetime,study_site,study_site_name,hostname_created
0,066-31760001-4,F,1982-01-09 00:00:00+00:00,2017-09-06 08:14:48+00:00,2017-09-06 00:00:00+00:00,C0,06676HU8EE02013206,06676HU8EE00000201,06676HU8EE,76HU8EE,...,Plasma,PL,32,2017-09-07 00:00:00+00:00,False,Research Blood Draw,2017-09-06 08:16:47+00:00,31,mathangwane,bcpp076
1,066-31760002-2,F,1972-06-12 00:00:00+00:00,2017-09-12 07:50:33+00:00,2017-09-12 00:00:00+00:00,C0,066768VWFG02013206,066768VWFG00000201,066768VWFG,768VWFG,...,Plasma,PL,32,2017-09-12 00:00:00+00:00,False,Research Blood Draw,2017-09-12 07:58:25+00:00,31,mathangwane,bcpp076
2,066-31760003-0,F,1965-01-01 00:00:00+00:00,2017-09-13 08:11:15+00:00,2017-09-13 00:00:00+00:00,C0,06676YMWBV02011602,06676YMWBV00000201,06676YMWBV,76YMWBV,...,Buffy Coat,BC,16,2017-09-13 00:00:00+00:00,False,Viral Load,2017-09-13 08:24:28+00:00,31,mathangwane,bcpp076
3,066-31760002-2,F,1972-06-12 00:00:00+00:00,2017-09-12 07:50:33+00:00,2017-09-12 00:00:00+00:00,C0,066768VWFG02011602,066768VWFG00000201,066768VWFG,768VWFG,...,Buffy Coat,BC,16,2017-09-12 00:00:00+00:00,False,Research Blood Draw,2017-09-12 07:58:25+00:00,31,mathangwane,bcpp076
4,066-31760003-0,F,1965-01-01 00:00:00+00:00,2017-09-13 08:11:15+00:00,2017-09-13 00:00:00+00:00,C0,06676YMWBV02011603,06676YMWBV00000201,06676YMWBV,76YMWBV,...,Buffy Coat,BC,16,2017-09-13 00:00:00+00:00,False,Viral Load,2017-09-13 08:24:28+00:00,31,mathangwane,bcpp076


In [13]:
# export requisitions to CSV as a single file
path = f'~/lab_requisitions_{project}_{timestamp}.csv'
df_requisition.to_csv(path, index=True, date_format=export_date_format, sep=delimiter)
sys.stdout.write(f'* {path}\n')

* ~/lab_requisitions_bcpp_clinic_20171014093503.csv


In [14]:
# export aliquots to CSV as a single file
path = f'~/lab_aliquots_{project}_{timestamp}.csv'
df_aliquot.to_csv(path, index=True, date_format=export_date_format, sep=delimiter)
sys.stdout.write(f'* {path}\n')

* ~/lab_aliquots_bcpp_clinic_20171014093503.csv


In [15]:
df_aliquot[df_aliquot['missing_requisition']]

,subject_identifier,gender,dob,visit_datetime,visit_date,visit_code,aliquot_identifier,parent_identifier,identifier_prefix,requisition_identifier,...,aliquot_type,alpha_code,numeric_code,aliquot_date,missing_requisition,panel_name,requisition_datetime,study_site,study_site_name,hostname_created


In [16]:
print(df_aliquot.groupby('medium').size())
print('---')

print(df_aliquot.groupby('aliquot_type').size())
print('---')

print(df_aliquot.groupby('alpha_code').size())
print('---')

print(df_aliquot.groupby('numeric_code').size())
print('---')

print(df_aliquot.groupby('condition').size())
print('---')

print(df_aliquot.groupby('missing_requisition').size())
print('---')

medium
tube    133
dtype: int64
---
aliquot_type
Buffy Coat     38
Plasma         76
Whole Blood    19
dtype: int64
---
alpha_code
BC    38
PL    76
WB    19
dtype: int64
---
numeric_code
02    19
16    38
32    76
dtype: int64
---
condition
10    133
dtype: int64
---
missing_requisition
False    133
dtype: int64
---
